# Metryki klasyfikacji

Aby poznać popularne metody oceny klasyfikatorów wygenerujemy sobie przykładowe sekwencje klas przykładów. Wykorzystamy w tym celu bibliotekę numpy oraz generator liczb losowych z rozkładu normalnego.

In [1]:
import numpy as np

In [2]:
from numpy.random import RandomState

In [5]:
random = RandomState(13)

random_1 = random.normal(loc=0.0, size=100)
random_2 = random.normal(loc=0.5, size=100)
y_test = [1 if i >= 0 else 0 for i in random_1]
y_pred = [1 if i >= 0 else 0 for i in random_2]
print(y_test[:10])
print(y_pred[:10])

[0, 1, 0, 1, 1, 1, 1, 1, 1, 0]
[0, 0, 0, 1, 1, 1, 0, 1, 0, 1]


## Macierz błędów

Pierwszym krokiem będzie stworzenie macierzy błędów, czyli wyznaczenia true postives, true negatives, false positives i false negatives.

In [6]:
def confusion_matrix(truth, prediction):
    tp, tn, fp, fn = 0, 0, 0, 0
    for label, predicted in zip(truth, prediction):
        if label == predicted:
            if predicted:
                tp += 1
            else:
                tn += 1
        else:
            if predicted:
                fp += 1
            else:
                fn += 1
    return tp, tn, fp, fn

In [7]:
# zastosowanie dla 
confusion_matrix(y_test, y_pred)

(33, 21, 30, 16)

## Accuracy

Pierwszą miarą jest miara dokładności 

In [8]:
def accuracy(truth, prediction):
    tp, tn, fp, fn = confusion_matrix(truth, prediction)
    return (tp + tn) / (tp + tn + fp + fn)

Wynik dla analizowanych predykcji

In [9]:
accuracy(y_test, y_pred)

0.54

Z accuracy związany jest problem - zachowuje się źle przy niezbalansowanych zbiorach. Wygenerujemy zbiór, w którym większość przykładów będzie pozytywna.

In [10]:
# żeby większość była 1
random = RandomState(13)

random_3 = random.normal(loc=-1, scale=0.5, size=100)
y_extr = [1 if i >= 0 else 0 for i in random_3]
sum(y_extr)

3

Nasz "klasyfikator" ma stosunkowo niską dokładność.

In [11]:
accuracy(y_extr, y_pred)

0.38

Zakładając z góry, że wszystkie przykłady są pozytywne uzyskujemy bardzo wysoką dokładność. Takie zachowanie jest w wielu przypadkach niepożądane - przypuśćmy, że próbujemy stworzyć klasyfikator do komórek rakowych, gdzie większość przypadków jest negatywna - dla takiego podejścia, zwrócenie informacji że wszystkie przypadki są negatywne da bardzo wysoką dokładność.

In [12]:
accuracy(y_extr, 100 * [0])

0.97

## Recall

Ta miara z kolei mówi o tym, jaką część dodatnich wyników wykrył klasyfikator

In [13]:
def recall(truth, prediction):
    tp, tn, fp, fn = confusion_matrix(truth, prediction)
    return tp /(tp + fn)

In [14]:
recall(y_test, y_pred)

0.673469387755102

In [15]:
recall(y_extr, y_pred)

0.6666666666666666

In [16]:
recall(y_extr, 100 * [0])

0.0

In [17]:
recall(y_extr, 100 * [1])

1.0

## Precision

Jest to miara, która skupia się tylko na przykładach pozytywnych - mówi jaka część wyników wskazanych przez klasyfikator jako dodatnie jest rzeczywiście dodatnia.

In [18]:
def precision(truth, prediction):
    tp, tn, fp, fn = confusion_matrix(truth, prediction)
    return tp / (tp + fp)
    

In [19]:
precision(y_test, y_pred)

0.5238095238095238

In [20]:
precision(y_extr, y_pred)

0.031746031746031744

In [21]:
precision(y_extr, 100 * [1])

0.03

# F1 Score

Jest to średnia harmoniczna precyzji i czułości. Ogólnie - im wyższy F1-score tym lepszy jest klasyfikator.

In [22]:
def f1score(truth, prediction):
    prec = precision(truth, prediction)
    rec = recall(truth, prediction)
    return 2 * prec * rec / (prec + rec)

In [23]:
f1score(y_test, y_pred)

0.5892857142857143

In [24]:
f1score(y_extr, y_pred)

0.06060606060606061

In [25]:
f1score(y_extr, 100 * [1])

0.058252427184466014